## №1

In [1]:
import numpy as np
from scipy.optimize import minimize

In [2]:
def func_min(x): 
    return sum(i**2 for i in x)

def func_max(x):
    return -sum(i**2 for i in x) 

constraints = {
    'type': 'eq', 
    'fun':lambda x: sum(i**4 for i in x) - 1
}

In [ ]:
n = 100
start = np.zeros(n, dtype=float)
result = minimize(func_min, start, method='trust-constr', constraints=constraints)


In [4]:
print('Минимум', result.fun)
print('Аргумент', result.x)

Минимум 0.0
Аргумент [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]


In [ ]:
result = minimize(func_max, start, method='trust-constr', constraints=constraints)


In [6]:
print('Максимум', result.fun)
print('Аргумент', result.x)

Максимум -9.999999999999982
Аргумент [0.31622777 0.31622777 0.31622777 0.31622775 0.31622775 0.31622775
 0.31622777 0.31622775 0.31622777 0.31622776 0.31622776 0.31622776
 0.31622777 0.31622777 0.31622775 0.31622776 0.31622776 0.31622776
 0.31622776 0.31622776 0.31622776 0.31622776 0.31622777 0.31622777
 0.31622777 0.31622777 0.31622777 0.31622777 0.31622777 0.31622777
 0.31622777 0.31622777 0.31622776 0.31622776 0.31622776 0.31622776
 0.31622777 0.31622777 0.31622778 0.31622778 0.31622777 0.31622777
 0.31622775 0.31622778 0.31622775 0.31622776 0.31622776 0.31622776
 0.31622776 0.31622777 0.31622777 0.31622778 0.31622777 0.31622778
 0.31622777 0.31622777 0.31622777 0.31622778 0.31622778 0.31622778
 0.31622776 0.31622778 0.31622776 0.31622776 0.31622778 0.31622777
 0.31622778 0.31622777 0.31622777 0.31622777 0.31622777 0.31622777
 0.31622777 0.31622776 0.31622776 0.31622776 0.31622778 0.31622776
 0.31622776 0.31622775 0.31622777 0.31622776 0.31622775 0.31622777
 0.31622779 0.31622776 0.

In [7]:
print(np.sqrt(n))
assert round(1 / n**(1/4), 4) == round(result.x[0], 4)
print('All right')

10.0
All right


## №2

In [20]:
import random as rn
import numpy as np
from numpy.random import choice as np_choice

class AntColony(object):

    def __init__(self, distances, n_ants, n_best, n_iterations, decay, alpha=1, beta=1):
        """
        Args:
            distances (2D numpy.array): Square matrix of distances. Diagonal is assumed to be np.inf.
            n_ants (int): Number of ants running per iteration
            n_best (int): Number of best ants who deposit pheromone
            n_iteration (int): Number of iterations
            decay (float): Rate it which pheromone decays. The pheromone value is multiplied by decay, so 0.95 will lead to decay, 0.5 to much faster decay.
            alpha (int or float): exponenet on pheromone, higher alpha gives pheromone more weight. Default=1
            beta (int or float): exponent on distance, higher beta give distance more weight. Default=1

        Example:
            ant_colony = AntColony(german_distances, 100, 20, 2000, 0.95, alpha=1, beta=2)          
        """
        self.distances  = distances
        self.pheromone = np.ones(self.distances.shape) / len(distances)
        self.all_inds = range(len(distances))
        self.n_ants = n_ants
        self.n_best = n_best
        self.n_iterations = n_iterations
        self.decay = decay
        self.alpha = alpha
        self.beta = beta

    def run(self):
        shortest_path = None
        all_time_shortest_path = ("placeholder", np.inf)
        for i in range(self.n_iterations):
            all_paths = self.gen_all_paths()
            self.spread_pheronome(all_paths, self.n_best, shortest_path=shortest_path)
            shortest_path = min(all_paths, key=lambda x: x[1])
            if shortest_path[1] < all_time_shortest_path[1]:
                all_time_shortest_path = shortest_path            
            self.pheromone = self.pheromone * self.decay            
        return all_time_shortest_path

    def spread_pheronome(self, all_paths, n_best, shortest_path):
        sorted_paths = sorted(all_paths, key=lambda x: x[1])
        for path, dist in sorted_paths[:n_best]:
            for move in path:
                self.pheromone[move] += 1.0 / self.distances[move]

    def gen_path_dist(self, path):
        total_dist = 0
        for ele in path:
            total_dist += self.distances[ele]
        return total_dist

    def gen_all_paths(self):
        all_paths = []
        for i in range(self.n_ants):
            path = self.gen_path(0)
            all_paths.append((path, self.gen_path_dist(path)))
        return all_paths

    def gen_path(self, start):
        path = []
        visited = set()
        visited.add(start)
        prev = start
        for i in range(len(self.distances) - 1):
            move = self.pick_move(self.pheromone[prev], self.distances[prev], visited)
            path.append((prev, move))
            prev = move
            visited.add(move)
        path.append((prev, start))    
        return path

    def pick_move(self, pheromone, dist, visited):
        pheromone = np.copy(pheromone)
        pheromone[list(visited)] = 0

        row = pheromone ** self.alpha * (( 1.0 / dist) ** self.beta)

        norm_row = row / row.sum()
        move = np_choice(self.all_inds, 1, p=norm_row)[0]
        return move


In [21]:
matrix = np.array([
    [np.inf, 4, 5, 7, 5],
    [8, np.inf, 5, 6, 6],
    [3, 5, np.inf, 9, 6],
    [3, 5, 6, np.inf, 2],
    [6, 2, 3, 8, np.inf]
])

In [24]:
ant_colony = AntColony(matrix, 100, 1, 100, 0.95, alpha=1, beta=1)
shortest_path = ant_colony.run()
print(f'Path: {shortest_path[0]}')
print(f'Cost: {shortest_path[1]}')

Path: [(0, 1), (1, 3), (3, 4), (4, 2), (2, 0)]
Cost: 18.0


Как видим, пути совпадают!

В муравьином алгоритме отстуствует начальное положение, поэтому проинтерпретируем количество эпох.
\
Эпоха в муравьином алгоритме представляет собой один проход всеми муравьями через граф задачи.
\
Увеличение числа эпох может повысить вероятность нахождения оптимального решения, так как муравьи имеют больше шансов исследовать пространство решений.
\
Однако слишком большое количество эпох может привести к увеличению времени выполнения алгоритма.